# Assignement 1 study of Master data from IS Academia

## useful imports

In [179]:
import pandas as pd
import scipy.stats as stats
import numpy as np

## Import Master data

Reading the pickle with the master data frame

In [144]:
s1 = master_df.xs('Master semestre 1', level=1).reset_index(0)
s1 = s1[s1.Statut == 'Présent']

pa = master_df.xs('Projet Master automne', level=1).reset_index(0)

#pa = pa[pa.Statut == 'Présent'].reset_index(0)

pp = master_df.xs('Projet Master printemps', level=1).reset_index(0)

#pp = pp[pp.Statut == 'Présent'].reset_index(0)

master_sciper = s1.ix[s1.index & (pa.index | pp.index)].index

master_s1_papp = master_df.loc[master_df.index.get_level_values("Sciper").isin(master_sciper.values)]

In [145]:
master_s1_papp.shape

(486, 9)

In [146]:
master_semesters = []
scipers = master_s1_papp.index.get_level_values(2).drop_duplicates()
print(scipers.shape)
for sciper in scipers:
    student = master_s1_papp.xs(sciper, level=2)
    num_sems = student['Statut'].count()
    civ = student['Civilité'][0]
    spec = student['Spécialisation'][0]
    master_semesters.append([sciper, civ, num_sems, spec])

master_semesters = pd.DataFrame(master_semesters,columns=['Sciper', 'Civilité', 'Number of semesters', 'Spécialisation'])

(114,)


In [147]:
master_semesters = master_semesters.fillna('aucune')

In [151]:
spec_count = master_semesters.groupby('Spécialisation')['Spécialisation'].count()
sems_mean = master_semesters.groupby('Spécialisation')['Number of semesters'].mean()
sems_stats = pd.concat([spec_count,sems_mean], axis=1, keys=['Count','Mean number of semesters'])
sems_stats

total mean :  4.26315789474


,Count,Mean number of semesters
Spécialisation,,
Biocomputing,1,6.000000
Computer Engineering - SP,2,5.000000
Foundations of Software,7,5.142857
Information Security - SP,2,5.000000
Internet computing,10,4.700000
Service science,1,5.000000
"Signals, Images and Interfaces",2,5.500000
Software Systems,2,5.000000
aucune,87,4.034483


In [187]:
total_mean = master_semesters['Number of semesters'].mean()
alpha = 0.05

for name, counts in master_semesters.groupby('Spécialisation')['Number of semesters']:
    test = stats.ttest_1samp(counts.values, total_mean)
    print("Results for {},".format(name), "the seperate means are :", counts.mean(), " and ", total_mean)
    print("Number of values :", len(counts.values))
    pvalue = test.pvalue
    
    if np.isnan(pvalue):
        print("not enough values to give a significant result")
    else:    
        print("Test p-value = %f" % pvalue)
        print("The means are statistically different") if test.pvalue < alpha else print("The means are statistically equal")
    print("")
        
        

Results for Biocomputing, the seperate means are : 6.0  and  4.26315789474
Number of values : 1
not enough values to give a significant result

Results for Computer Engineering - SP, the seperate means are : 5.0  and  4.26315789474
Number of values : 2
Test p-value = 0.000000
The means are statistically different

Results for Foundations of Software, the seperate means are : 5.14285714286  and  4.26315789474
Number of values : 7
Test p-value = 0.134314
The means are statistically equal

Results for Information Security - SP, the seperate means are : 5.0  and  4.26315789474
Number of values : 2
Test p-value = 0.000000
The means are statistically different

Results for Internet computing, the seperate means are : 4.7  and  4.26315789474
Number of values : 10
Test p-value = 0.070992
The means are statistically equal

Results for Service science, the seperate means are : 5.0  and  4.26315789474
Number of values : 1
not enough values to give a significant result

Results for Signals, Images

/usr/local/lib/python3.5/site-packages/numpy/core/_methods.py:82: RuntimeWarning: Degrees of freedom <= 0 for slice
  warnings.warn("Degrees of freedom <= 0 for slice", RuntimeWarning)
